This is a mock Jupyter workflow of a RNA Seq process based on the following requirements:
*Raw FASTA data 
*Run quality control with FASTQC 
*Use python cutadapt to cut out low quality regions and adapters 
*Use HISAT to align to genome/StringTIE to align genomes and transcriptomes → later 
*Visualize StringTIE reads using UC Santa Cruz viz tool 
*Convert to count tables 
*DESeq2 → R code that does differential expression 
*BAM files


In [5]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import AgentExecutor
from langchain_experimental.tools import PythonREPLTool
from langchain.agents import create_openai_functions_agent
import getpass
import os

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [7]:
tools = [PythonREPLTool()]
instructions = """You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code. You are to use a variety of bioinformatics packages.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.
"""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [11]:
agent = create_openai_functions_agent(ChatOpenAI(temperature=0), tools, prompt)
agentexecutor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [13]:
agentexecutor.invoke({"input": "Could you open the FASTA file data/GCF_020097155.1_ASM2009715v1_genomic.fa and give me the first row?"})



> Entering new AgentExecutor chain...

Invoking: `Python_REPL` with `from Bio import SeqIO
record = next(SeqIO.parse('data/GCF_020097155.1_ASM2009715v1_genomic.fa', 'fasta'))
record`


The first row of the FASTA file is:
```
ID: NC_025697.1
Name: NC_025697.1
Description: NC_025697.1 Vibrio cholerae O1 biovar El Tor str. N16961 chromosome 1, complete sequence
Number of features: 0
Seq('AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAG...TTC', SingleLetterAlphabet())
```

> Finished chain.


{'input': 'Could you open the FASTA file data/GCF_020097155.1_ASM2009715v1_genomic.fa and give me the first row?',
 'output': "The first row of the FASTA file is:\n```\nID: NC_025697.1\nName: NC_025697.1\nDescription: NC_025697.1 Vibrio cholerae O1 biovar El Tor str. N16961 chromosome 1, complete sequence\nNumber of features: 0\nSeq('AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAG...TTC', SingleLetterAlphabet())\n```"}

Can open a FASTA file. Nex step, can it do quality control analysis?